# Data Wrangling with Spark SQL Quiz

In [4]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) instantiate a Spark session 
# 2) read in the data set located at the path "data/sparkify_log_small.json"
# 3) create a view to use with your SQL queries
# 4) write code to answer the quiz questions 

spark = SparkSession \
    .builder \
    .appName("Spark SQL Quiz") \
    .getOrCreate()

user_log = spark.read.json("data/sparkify_log_small.json")

user_log.createOrReplaceTempView("log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [13]:
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [8]:
spark.sql("SELECT page\
            FROM log_table\
            WHERE userID=''").show()

+-----+
| page|
+-----+
|Login|
| Home|
| Home|
| Home|
|Login|
|Login|
| Home|
| Home|
|Login|
| Home|
|Login|
| Home|
| Help|
| Home|
|Login|
| Home|
|Login|
| Home|
|Login|
| Home|
+-----+
only showing top 20 rows



In [12]:
spark.sql("SELECT distinct page\
            FROM log_table\
            WHERE page NOT IN (\
            SELECT page\
            FROM log_table\
            WHERE userID='')").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 3

How many female users do we have in the data set?

In [17]:
spark.sql("SELECT count(distinct userId)\
            FROM log_table\
            WHERE gender='F'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [32]:
spark.sql("SELECT artist, plays \
            FROM (SELECT artist, COUNT(*) as plays \
            FROM log_table \
            WHERE artist is not NULL\
            GROUP BY artist) AS M \
            ORDER BY plays desc \
            LIMIT 1").show()

+--------+-----+
|  artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [68]:
M1 = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_HOME\
            FROM log_table\
            WHERE (page='Home') or (page='NextSong')\
            ")

M1.createOrReplaceTempView("M1")


M2 = spark.sql("SELECT *, sum(is_HOME)\
            OVER (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as stayTime\
           FROM M1\
          ")
M2.createOrReplaceTempView("M2")

spark.sql("SELECT avg(cnts)\
          FROM (SELECT COUNT(*) as cnts\
          FROM M2\
          GROUP BY userID, stayTime, page\
          HAVING page='NextSong')").show()

+-----------------+
|        avg(cnts)|
+-----------------+
|6.898347107438017|
+-----------------+

